<a href="https://colab.research.google.com/github/29315668/Brain-Tumour-Detection-and-classification/blob/master/Exe1TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=7cdc0517042c490b3b189ab4208316c8be450531bd2769ef96e91d772e4c0189
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as pyspark_functions
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("EX1").getOrCreate()
sc = spark.sparkContext

In [ ]:

bus = sc.textFile("/content/bus.txt")
bus.take(5)

['2015-2016,Special Ed AM Run,48186,N758,Other,2015-09-02T06:27:00,0',
 '2015-2016,Special Ed AM Run,2518,L530,Mechanical Problem,2015-09-02T06:24:00,0',
 '2015-2016,Special Ed AM Run,235,K168,Other,2015-09-02T06:45:00,0',
 '2015-2016,Special Ed AM Run,2102,K216,Other,2015-09-02T06:55:00,1',
 '2015-2016,Special Ed AM Run,48162,N861,Mechanical Problem,2015-09-02T06:55:00,0']

In [ ]:
reasonCount=bus.map(lambda x:x.split(",")[4]).countByValue()
topreason=sorted(reasonCount.items(),key=lambda x:-x[1])[:5]
print(topreason)

[('Heavy Traffic', 170660), ('Other', 37019), ('Mechanical Problem', 27821), ('Won`t Start', 12172), ('Flat Tire', 8307)]


In [ ]:
#methode 1
routCount=bus.map(lambda x:x.split(",")[3]).countByValue()
toprout=sorted(routCount.items(),key=lambda x:-x[1])[:5]
print(toprout)

[('1', 3879), ('2', 2933), ('5', 2530), ('3', 2486), ('4', 1480)]


In [ ]:
#methode 2
routeCount=bus.map(lambda x:(x.split(",")[3],1)).reduceByKey(lambda x,y:x+y)
topRoute=sorted(routeCount.collect(),key=lambda x:-x[1])[:5]
print(topRoute)

[('1', 3879), ('2', 2933), ('5', 2530), ('3', 2486), ('4', 1480)]


In [ ]:
incident_with_students=bus.filter(lambda x:int(x.split(",")[6])>0)
nbrStd = incident_with_students.count()
print(nbrStd)

109276


In [ ]:
incident_with_students=bus.filter(lambda x:int(x.split(",")[6])==0)
nbrStd = incident_with_students.count()
print(nbrStd)

167837


In [ ]:
#methode 1 question 5
yearAccident=bus.map(lambda x:x.split(",")[0]).countByValue()
moinAccident=sorted(yearAccident.items(),key=lambda x:x[1])[:1]
print(moinAccident)

[('2019-2020', 1)]


In [ ]:
#methode 2 question 5
year=bus.map(lambda x:(x.split(",")[0],1)).reduceByKey(lambda x,y:x+y)
topYear=year.min(lambda x:x[1])
print(topYear)

('2019-2020', 1)


***Exercice 2***

In [6]:
chicago=sc.textFile("/content/chicago_crime_2016.csv")

In [7]:
header=chicago.first()
crimes=chicago.filter(lambda x: x!=header)

In [8]:
crime_count=crimes.map(lambda x:(x.split(",")[5],1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:-x[1])
crime_count.take(5)

[('THEFT', 49603),
 ('BATTERY', 41950),
 ('CRIMINAL DAMAGE', 25573),
 ('ASSAULT', 15631),
 ('OTHER OFFENSE', 13971)]

# Analyse des tendances des différents types de crimes au fil des mois

In [12]:
monthly_crime_counts = crimes.map(lambda crime: (crime.split(",")[2].split(" ")[0] .split("/")[0], 1)).reduceByKey(lambda x, y: x + y).sortByKey().collect()

print("Monthly crime counts:")
print(monthly_crime_counts)


Monthly crime counts:
[('01', 20301), ('02', 18548), ('03', 21847), ('04', 20845), ('05', 23132), ('06', 23574), ('07', 24338), ('08', 24344), ('09', 22982), ('10', 17379)]


# les trois principaux endroits  où les taux de crimes sont les plus élevés.


In [18]:
location_crime_counts = crimes.map(lambda crime: (crime.split(",")[7], 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: -x[1]).take(3)

 # Print the top 3 crime locations and their incident counts

for location, count in location_crime_counts:
      print(f"Location: {location},Incidents: (count)")

Location: STREET,Incidents: (count)
Location: RESIDENCE,Incidents: (count)
Location: APARTMENT,Incidents: (count)


# les trois principaux types de crimes en fonction du nombre d'incidents dans la zone est "RESIDENCE" en utilisant "Location Description"

In [20]:
residence_crimes =crimes.filter(lambda crime: crime.split(",")[7].upper()=="RESIDENCE")

crime_type_counts_residence= residence_crimes.map(lambda crime: (crime.split(",")[5], 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: -x[1]).take(3)

print(crime_type_counts_residence)


[('BATTERY', 8178), ('OTHER OFFENSE', 5417), ('CRIMINAL DAMAGE', 4563)]


# Le poucentage de crimes qui ménent à une arrestation

In [25]:
total_crimes=crimes.count()
arrested_crimes= crimes.filter(lambda crime: crime.split(",")[8]=="True").count()
arrest_rate = (arrested_crimes/ total_crimes)*100
print("Arrested rate:", arrest_rate)

Arrested rate: 0.0


# Les jours de la semaine ou les taux de crimes sont les plus élevés et les plus faibles

In [30]:
day_of_week_crime_counts =crimes.map(lambda crime: (crime.split(",")[2].split(" ") [0] , 1)).reduceByKey(lambda x, y: x+y)
higher_day_of_week_crime= day_of_week_crime_counts.max(lambda x: x[1])
lower_day_of_week_crime =day_of_week_crime_counts.min(lambda x: x[1])
print (f"day with the highest crime rate: {higher_day_of_week_crime[0]}")
print (f"day with the lowest crime rate: {lower_day_of_week_crime[0]}")

day with the highest crime rate: 07/30/2016
day with the lowest crime rate: 01/18/2016


# Le pourcentage d'affaires domestiques pour différents types de crimes

In [33]:
domestic_crimes = crimes.filter(lambda crime: crime.split(",")[9] == "true").count()
non_domestic_crimes = total_crimes - domestic_crimes
domestic_percentage = (domestic_crimes / total_crimes)*100
print("Percentage of domestic incidents:", domestic_percentage)

Percentage of domestic incidents: 16.303557457775323


# Enquéter si certains types est plus couronts au cours de saisons ou de mois précis

In [47]:
monthly_crime_counts = crimes.map(lambda crime: (crime.split(",")[2].split(" ")[0] .split("/")[0], 1)).reduceByKey(lambda x, y: x + y).sortByKey(lambda x: -x[1]).take(1)
print("Month with the most crimes:", monthly_crime_counts)

Month with the most crimes: [('01', 20301)]
